**This is the final pre-processing code for the region insights in terms of foundational cleaning and merging**

In [1]:
import pandas as pd
import os
import re
from datetime import datetime as dt
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests
from api_keys import (gkey, gkey2, gkey3)
import time
import random
import pickle
import json

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Merging the Data to a single DF

In [2]:
def find_all_files(path, ty='csv', Name='Merged_DF.csv'):
    #Iteratively appends all files with ty extention to list_of_files
    for root,dirs,files in os.walk(path):
        [list_of_files.append(file) for file in files if (file.endswith(f".{ty}") and (file!=Name))]

In [3]:
Name = 'Merged_DF.csv' #Name of Final DF
list_of_files = []
data_path = os.path.join('..', 'Data')
find_all_files(data_path, Name=Name)

Total_DF = pd.DataFrame()
for file in list_of_files:
    try:
        DF = pd.read_csv(os.path.join('..', 'Data', file), encoding='utf-8')
    except:
        DF = pd.read_csv(os.path.join('..', 'Data', file), encoding='latin1')
    DF['country'] = file[:2]
    Total_DF = (DF if Total_DF.empty else pd.concat([Total_DF, DF]))

# Total DF Cleaning

In [4]:
#Total_DF['video_id'].map(lambda x: type(x)!=str).sum() #All values are string
#Total_DF['trending_date'].map(lambda x: type(x)!=str).sum() #All values are string
#Convert the 'trending_date' to date format
Total_DF['trending_date'] = Total_DF['trending_date'].map(lambda x: dt.strptime(x, "%y.%d.%m"))

#Total_DF['title'].map(lambda x: type(x)!=str).sum() #All values are string
#Total_DF['channel_title'].map(lambda x: type(x)!=str).sum() #All values are string
#Total_DF['category_id'].map(lambda x: type(x)!=int).sum() #All values are int
#Total_DF['publish_time'].map(lambda x: type(x)!=str).sum() #All values are string
#Convert the 'publish_time' to date format
Total_DF['publish_time'] = pd.to_datetime(Total_DF['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ') #%f means microsecond which means 6 digits. This works here as it is always 0 microseconds
#Total_DF['tags'].map(lambda x: type(x)!=str).sum() #All values are string
#Total_DF['views'].map(lambda x: type(x)!=int).sum() #All values are int64
#Total_DF['likes'].map(lambda x: type(x)!=int).sum() #All values are int64
#Total_DF['dislikes'].map(lambda x: type(x)!=int).sum() #All values are int64
#Total_DF['comment_count'].map(lambda x: type(x)!=int).sum() #All values are int64
#Total_DF['likes'].isnull().sum()
#Total_DF['dislikes'].isnull().sum()
#Total_DF['comment_count'].isnull().sum()
#Total_DF['thumbnail_link'].isnull().sum()
#Total_DF['comments_disabled'].map(lambda x: type(x)!=bool).sum() #All values are boolean
#Total_DF['ratings_disabled'].map(lambda x: type(x)!=bool).sum() #All values are boolean
#Total_DF['video_error_or_removed'].map(lambda x: type(x)!=bool).sum() #All values are boolean
#Total_DF['comments_disabled'].isnull().sum()
#Total_DF['ratings_disabled'].isnull().sum()
#Total_DF['video_error_or_removed'].isnull().sum()

#Convert NaN values in 'description' to ''
Total_DF['description'].fillna(value='', inplace=True)
#Total_DF['description'].isna().sum()
#Total_DF[Total_DF['description'] == ''].shape

## Removing for duplicate rows

In [5]:
#Drop the duplicate rows
Total_DF.drop_duplicates(subset=['video_id', 'trending_date', 'country'], keep='last', inplace=True)#NEED TO CHANGE
Total_DF.reset_index(drop=True, inplace=True)
to_drop = Total_DF[(Total_DF['video_id']=='#NAME?') | (Total_DF['video_id']=='#VALUE!')].index
Total_DF.drop(to_drop, inplace=True)
Total_DF.reset_index(drop=True, inplace=True)
Total_DF.shape
#375942 - 14518 - 846  = 360578

(360578, 17)

## Removing Videos with multiple publish Times and videos with 'video_error_or_removed'

In [6]:
#Publish time is supposed to be unique. Remove the videos with More than 1 publish time 
Temp_TF = Total_DF.groupby('video_id').aggregate(Publish_Time_Unique_Count = ('publish_time', lambda x : len(set(x.to_list()))))           
#Drop these 29 videos, total 146 corresponding rows
Total_DF.drop(Total_DF[Total_DF['video_id'].isin(Temp_TF[Temp_TF['Publish_Time_Unique_Count']>1].index)].index, inplace=True)
Total_DF.reset_index(drop=True, inplace=True)
Total_DF.shape
#360578 - 146 = 360432

#Some Videos are removed after some time; Let's exclude these videos from the analysis as there is a manual intervention or environment issue 
#50 videos had error (atleast once); Total 215 rows
#Both the below codes give exactly same results; MEANS 'video_error_or_removed' really means that atleast once 'video_error_or_removed'
#Total_DF[Total_DF['video_id'].isin(Total_DF[Total_DF['video_error_or_removed']]['video_id'].unique())]
#Total_DF[Total_DF['video_error_or_removed']]
Total_DF.drop(Total_DF[Total_DF['video_id'].isin(Total_DF[Total_DF['video_error_or_removed']]['video_id'].unique())].index, inplace=True)
Total_DF.reset_index(drop=True, inplace=True)
Total_DF.shape
#360432-215 = 360217

#Remove 'video_error_or_removed' as it doesn't carry any relevant info now
Total_DF.drop('video_error_or_removed', inplace=True, axis=1)
Total_DF.shape

(360432, 17)

(360217, 17)

(360217, 16)

# Save the DataFrame as pickle 

In [7]:
#Total_DF.to_pickle("../Data/VideoDF.pkl") #Commenting to avoid overwritting the existing file

# Read from pickle

In [8]:
Video_DF = pd.read_pickle("../Data/VideoDF.pkl")

In [9]:
Video_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360217 entries, 0 to 360216
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   video_id           360217 non-null  object        
 1   trending_date      360217 non-null  datetime64[ns]
 2   title              360217 non-null  object        
 3   channel_title      360217 non-null  object        
 4   category_id        360217 non-null  int64         
 5   publish_time       360217 non-null  datetime64[ns]
 6   tags               360217 non-null  object        
 7   views              360217 non-null  int64         
 8   likes              360217 non-null  int64         
 9   dislikes           360217 non-null  int64         
 10  comment_count      360217 non-null  int64         
 11  thumbnail_link     360217 non-null  object        
 12  comments_disabled  360217 non-null  bool          
 13  ratings_disabled   360217 non-null  bool    

In [10]:
#Checking Sanity
Video_DF.dtypes
Video_DF.duplicated(subset=['video_id', 'trending_date', 'country'], keep='last').sum()

video_id                     object
trending_date        datetime64[ns]
title                        object
channel_title                object
category_id                   int64
publish_time         datetime64[ns]
tags                         object
views                         int64
likes                         int64
dislikes                      int64
comment_count                 int64
thumbnail_link               object
comments_disabled              bool
ratings_disabled               bool
description                  object
country                      object
dtype: object

0

##  Intruducing the Notion of Popularity. What makes trending videos popular ?

**Popularity score (Longevity): Total days of trending for a video**

**Populrity score of a trending video depends on what ?**

**Useful metric to consider from the dataset**
- Views of Trend Day 1
- Likes of Trend Day 1
- Dislikes of Trend Day 1
- Comment_Count of Trend Day 1
- Words in Title
- Channel Title
- Category ID
- Tags
- comments_disabled
- ratings_disabled
- video_error_or_removed
- description

**Extract the info outside dataset**
- Publish time of the day based on the timezone of the channel (Need outside info)
- Country
- Language

**Extract info outside news**
- News effect on popularity (Can be speific to a category)
- Google trending effect on popularity


# New Video Features via API call

### Sample API call

base = 'https://www.googleapis.com/youtube/v3/videos'


params = {
            'part' : 'snippet,contentDetails',
            #'part' :'localizations',
            #'part' : 'player',
            #'part' : 'recordingDetails',
            #'part' : 'statistics',
            #'part' : 'status',
            'id': <List of Videos>,
            'key':gkey
}

response = requests.get(base, params)


In [124]:
start_time = time.time()
#Video_ID_List = list(set(Video_DF['video_id'])) #COMMENTING NOT TO ACCEDENTALLY RUN THE ENTIRE BATCH
#Video_ID_List = pickle.load(open( "To_scrape.p", "rb" )) #THIS IS USED TO SCRAPE MISSED VIDEOS

Total = []
len(Video_ID_List)
Video_features = dict()
base = 'https://www.googleapis.com/youtube/v3/videos'
Total_Videos = len(Video_ID_List)
count = 0
for iteration in range(((len(Video_ID_List)//50 + 1) if len(Video_ID_List)%50 else len(Video_ID_List)//50)):
    list_of_videos = (Video_ID_List[iteration*50:iteration*50+50] if (iteration+1)*50<=Total_Videos else Video_ID_List[iteration*50:])
    Total = Total + list_of_videos
    params = {
            'part' : 'snippet,contentDetails',
            'id': list_of_videos,
            'key':gkey
    }
    
    response = requests.get(base, params)
    
    try:
        content = response.json()
        
        for item in content['items']:
            count+=1
            ID = item['id']

            Video_features[ID] = {'ChannelID' : (item['snippet']['channelId'] if 'channelId' in item['snippet'] else None)}

            Video_features[ID].update({'ChannelTitle':(item['snippet']['channelTitle'] if 'channelTitle' in item['snippet'] else None)})

            Video_features[ID].update({'DefaultLanguage' : (item['snippet']['defaultLanguage'] if 'defaultLanguage' in item['snippet'] else None)})

            Video_features[ID].update({'DefaultAudioLanguage' : (item['snippet']['defaultAudioLanguage'] if 'defaultAudioLanguage' in item['snippet'] else None)})

            Video_features[ID].update({'Duration' : (item['contentDetails']['duration'] if 'duration'in item['contentDetails'] else None)})

            Video_features[ID].update({'Caption' : (item['contentDetails']['caption'] if 'caption'in item['contentDetails'] else None)})

            Video_features[ID].update({'RegionRestriction_Blocked' : (((item['contentDetails']['regionRestriction']['blocked']  if  ('blocked' in item['contentDetails']['regionRestriction']) else None)) if ('regionRestriction' in item['contentDetails']) else None)})   

            Video_features[ID].update({'RegionRestriction_Allowed' : (((item['contentDetails']['regionRestriction']['allowed']  if  ('allowed' in item['contentDetails']['regionRestriction']) else None)) if ('regionRestriction' in item['contentDetails']) else None)})   
            
    except:
        print(response.url)
        print(response)
        print(f"Didn't get response for iteration {iteration}")
        
    time.sleep(2)
    print(count)
time_taken_in_min=(time.time()-start_time)//60 
#Before I start,
#YouTube Data API v3	74	
    

115

25
51
59


## Saving the scraped Data

In [32]:
#Scraped_DF = pd.DataFrame(Video_features).transpose()
#Remove duplicates if any
#Scraped_DF=Scraped_DF[~Scraped_DF.duplicated(['video_id'], keep='first')]
#Scraped_DF.to_pickle("../Data/API_RETRIEVED_DATA.pkl")
Scraped_DF = pd.read_pickle('../Data/API_RETRIEVED_DATA.pkl')
Scraped_DF.shape

(124848, 9)

## Merge the Video Data with Scraped Data (left)

In [33]:
Video_DF.shape
Scraped_DF.shape

Video_DF.columns
Scraped_DF.columns

New_Video_DF = Video_DF.merge(Scraped_DF, how='left', on='video_id')
New_Video_DF.columns
New_Video_DF.shape

(360217, 16)

(124848, 9)

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'description', 'country'],
      dtype='object')

Index(['video_id', 'ChannelID', 'ChannelTitle', 'DefaultLanguage',
       'DefaultAudioLanguage', 'Duration', 'Caption',
       'RegionRestriction_Blocked', 'RegionRestriction_Allowed'],
      dtype='object')

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'description', 'country', 'ChannelID', 'ChannelTitle',
       'DefaultLanguage', 'DefaultAudioLanguage', 'Duration', 'Caption',
       'RegionRestriction_Blocked', 'RegionRestriction_Allowed'],
      dtype='object')

(360217, 24)

In [35]:
#New_Video_DF.to_pickle("../Data/New_Video_DF_360217.pkl")